# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import zipfile
import getpass
from time import sleep
from random import randint
import spotipy
from config import *

# Spotify API

## Token

In [121]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id = Client_ID, client_secret = Client_Secret)

In [122]:
cache_token = token.get_access_token()

C:\Users\alfon\AppData\Local\Temp\ipykernel_12628\4134625355.py:1: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  cache_token = token.get_access_token()


In [123]:
cache_token

{'access_token': 'BQA8Zgx4cee89ZY0STLtr828lu7MsGJwzFw9V7mpRGR8nsYhQ52l9IIJUQu0xu1uDB7rTHqht5K85Wj3HOrVLOJICKvEWP2vGXwB7xQgPaY1DxpROZc',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'expires_at': 1661266332}

In [124]:
sp = spotipy.Spotify(cache_token["access_token"])

## Request

In [62]:
results = sp.user_playlist_tracks("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf")

In [33]:
results["items"][0]["track"]["external_urls"]["spotify"]

'https://open.spotify.com/track/3AhXZa8sUQht0UEdBJgpGc'

### Function to get the data from the first 100 songs

In [76]:
def analyze_playlist(creator, playlist_id, pages=5, start=0):
    
    playlist_columns = ["Song", "Artist", "ID", "Link", "danceability", "energy", "key", "loudness", "mode", "speechiness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_columns)
    
    playlist = sp.user_playlist_tracks(creator, playlist_id, offset=start)
    nap = randint(1,2)
    for i in range(pages):
        print("Page",i+1)
        try:
            tracks = playlist["items"]
            for track in tracks:
                playlist_features = {}
                playlist_features["Song"] = track["track"]["name"]
                playlist_features["Artist"] = track["track"]["album"]["artists"][0]["name"]
                playlist_features["ID"] = track["track"]["id"]
                playlist_features["Link"] = track["track"]["external_urls"]["spotify"]
        
                audio_features = sp.audio_features(playlist_features["ID"])[0]
                for feature in playlist_columns[4:]:
                    playlist_features[feature] = audio_features[feature]
        
                track_df = pd.DataFrame(playlist_features, index = [0])
                playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
            playlist = sp.next(playlist)
            
        except:
            continue
        
        print("Sleeping for",nap,"seconds")
        sleep(nap)
    
    playlist_df.columns = ["Song", "Artist", "ID", "Link", "Danceability", "Energy", "Key", "Loudness", "Mode", "Speechiness", "Instrumentalness", "Liveness", "Valence", "Tempo", "Duration", "Time Signature"]
        
    return playlist_df

In [104]:
not_hot_0 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 5)
not_hot_0["ID"].nunique()

Page 1
Sleeping for 2 seconds
Page 2
Sleeping for 2 seconds
Page 3
Sleeping for 2 seconds
Page 4
Sleeping for 2 seconds
Page 5


467

In [105]:
skip1 = not_hot_0["ID"].nunique() + 1
skip1

468

In [106]:
not_hot_1 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 10, skip1)
not_hot_1["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2
Sleeping for 1 seconds
Page 3
Sleeping for 1 seconds
Page 4
Sleeping for 1 seconds
Page 5
Sleeping for 1 seconds
Page 6
Sleeping for 1 seconds
Page 7
Sleeping for 1 seconds
Page 8
Sleeping for 1 seconds
Page 9
Sleeping for 1 seconds
Page 10
Sleeping for 1 seconds


1000

In [107]:
skip2 = skip1 + not_hot_1["ID"].nunique() + 1
skip2

1469

In [108]:
not_hot_2 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 10, skip2)
not_hot_2["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2
Sleeping for 1 seconds
Page 3
Sleeping for 1 seconds
Page 4
Sleeping for 1 seconds
Page 5
Sleeping for 1 seconds
Page 6
Sleeping for 1 seconds
Page 7
Sleeping for 1 seconds
Page 8
Sleeping for 1 seconds
Page 9
Sleeping for 1 seconds
Page 10
Sleeping for 1 seconds


1000

In [109]:
skip3 = skip2 + not_hot_2["ID"].nunique() + 1
skip3

2470

In [110]:
not_hot_3 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 2, skip3)
not_hot_3["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2


188

In [111]:
skip4 = skip3 + not_hot_3["ID"].nunique() + 1
skip4

2659

In [112]:
not_hot_4 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 1, skip4)
not_hot_4["ID"].nunique()

Page 1


12

In [130]:
skip5 = skip4 + not_hot_4["ID"].nunique() + 1
skip5

2672

In [131]:
not_hot_5 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 4, skip5)
not_hot_5["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2
Sleeping for 1 seconds
Page 3
Sleeping for 1 seconds
Page 4


307

In [132]:
skip6 = skip5 + not_hot_5["ID"].nunique() + 1
skip6

2980

In [134]:
not_hot_6 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 5, skip6)
not_hot_6["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2
Sleeping for 1 seconds
Page 3
Sleeping for 1 seconds
Page 4
Sleeping for 1 seconds
Page 5


407

In [135]:
skip7 = skip6 + not_hot_6["ID"].nunique() + 1
skip7

3388

In [138]:
not_hot_7 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 1, skip7)
not_hot_7["ID"].nunique()

Page 1


82

In [139]:
skip8 = skip7 + not_hot_7["ID"].nunique() + 1
skip8

3471

In [140]:
not_hot_8 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 2, skip8)
not_hot_8["ID"].nunique()

Page 1
Sleeping for 1 seconds
Page 2


169

In [141]:
skip9 = skip8 + not_hot_8["ID"].nunique() + 1
skip9

3641

In [153]:
not_hot_9 = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf", 1, skip9)
not_hot_9["ID"].nunique()

Page 1


43

In [150]:
not_hot = pd.concat([not_hot_0, not_hot_1, not_hot_2, not_hot_3, not_hot_4, not_hot_5, not_hot_6, not_hot_7, not_hot_8, not_hot_9])
not_hot

,Song,Artist,ID,Link,Danceability,Energy,Key,Loudness,Mode,Speechiness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time Signature
0,Like a Rolling Stone,Bob Dylan,3AhXZa8sUQht0UEdBJgpGc,https://open.spotify.com/track/3AhXZa8sUQht0UE...,0.482,0.721,0,-6.839,1,0.0321,0,0.189,0.557,95.263,369600,4
1,Smells Like Teen Spirit,Nirvana,3oTlkzk1OtrhH8wBAduVEi,https://open.spotify.com/track/3oTlkzk1OtrhH8w...,0.485,0.863,1,-9.027,1,0.0495,0.0162,0.138,0.767,116.835,300977,4
2,A Day In The Life - Remastered,The Beatles,3ZFBeIyP41HhnALjxWy1pR,https://open.spotify.com/track/3ZFBeIyP41HhnAL...,0.364,0.457,4,-14.162,0,0.0675,0.000106,0.922,0.175,163.219,337413,4
3,Good Vibrations (Mono),The Beach Boys,5Qt4Cc66g24QWwGP3YYV9y,https://open.spotify.com/track/5Qt4Cc66g24QWwG...,0.398,0.413,1,-10.934,1,0.0388,0.000025,0.0891,0.331,133.574,219147,4
4,Johnny B Goode,Chuck Berry,7MH2ZclofPlTrZOkPzZKhK,https://open.spotify.com/track/7MH2ZclofPlTrZO...,0.518,0.756,10,-10.851,1,0.0915,0.000062,0.317,0.968,166.429,160893,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,I Don't Like,Chief Keef,1h6kgem1ai8vUgO1rZOwfB,https://open.spotify.com/track/1h6kgem1ai8vUgO...,0.742,0.844,2,-4.622,1,0.0479,0,0.0659,0.416,131.986,293840,4
39,"Happy - From ""Despicable Me 2""",Pharrell Williams,5b88tNINg4Q4nrRbrCXUmg,https://open.spotify.com/track/5b88tNINg4Q4nrR...,0.647,0.822,5,-4.662,0,0.183,0,0.0908,0.962,160.019,232720,4
40,Express Yourself - Remastered Version,Charles Wright & The Watts 103rd Street Rhythm...,6gQZKkphKIMxZgca5r7ImA,https://open.spotify.com/track/6gQZKkphKIMxZgc...,0.461,0.818,8,-8.482,1,0.125,0,0.266,0.721,92.305,229653,4
41,Call Me The Breeze,J.J. Cale,0D4JiHE8NkSQa5ZdZkYYLw,https://open.spotify.com/track/0D4JiHE8NkSQa5Z...,0.815,0.415,11,-15.573,1,0.0384,0.012,0.0943,0.817,91.539,157617,4


In [151]:
not_hot["ID"].nunique()

3675

In [152]:
not_hot.to_csv("Not_Hot.csv")

### Function to get the other 4900 songs

In [ ]:
results = sp.playlist_items("1G8IpkZKobrIlXcVPoSIuf")
next_pages = 49
artists = []
songs = []

for i in range(next_pages):
    tracks = sp.next(results)
    for y in range(0,100):
        try:
            track = tracks['items'][y]['track']['name']
            songs.append(track)
            artist = tracks['items'][y]['track']['artists'][0]['name']
            artists.append(artist)
        except:
            continue

df = pd.DataFrame({"Songs":songs, "Artists":artists})          
df

,Songs,Artists
0,Heroin,The Velvet Underground
1,Whole Lotta Shakin' Goin' On,Jerry Lee Lewis
2,Thunder Road,Bruce Springsteen
3,One,U2
4,I'm Waiting For The Man - Album Version (Stereo),The Velvet Underground
...,...,...
4895,Ms. Jackson,Outkast
4896,Walk on By,Dionne Warwick
4897,Folsom Prison Blues,Johnny Cash
4898,Sign 'O' the Times - Single Version,Prince


### Concat and save

In [313]:
not_hot = pd.concat([not_hot, df], ignore_index = True)
not_hot

,Songs,Artists
0,Like a Rolling Stone,Bob Dylan
1,Smells Like Teen Spirit,Nirvana
2,A Day In The Life - Remastered,The Beatles
3,Good Vibrations (Mono),The Beach Boys
4,Johnny B Goode,Chuck Berry
...,...,...
4995,Ms. Jackson,Outkast
4996,Walk on By,Dionne Warwick
4997,Folsom Prison Blues,Johnny Cash
4998,Sign 'O' the Times - Single Version,Prince


In [185]:
def search_song():
    print("Please write a song name:")
    song = input()
    results = sp.search(q=song, limit = 5)
    for track in results["tracks"]["items"]:
        print("Song:",track["name"])
        print("Artist:",track["artists"][0]["name"])
        print("Album:",track["album"]["name"])
        print("\n")

In [187]:
search_song()

Please write a song name:


 Californication


Song: Californication
Artist: Red Hot Chili Peppers
Album: Californication (Deluxe Edition)


Song: Californication
Artist: LAV8
Album: Covers Collection (Vol. 3)


Song: Scar Tissue
Artist: Red Hot Chili Peppers
Album: Californication (Deluxe Edition)


Song: Californication - Remix
Artist: Merkules
Album: Californication (Remix)


Song: Californication (Feels Like I'm Falling In Love)
Artist: Col3trane
Album: Lush Life




In [ ]:
song["tracks"]["items"][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [188]:
song["tracks"]["items"][0]["id"]

'0UuNO0yYvsMPlyLF7RfQlg'